
# Retrieval-Augmented Shopping Assistant - Embeddings Experimentations

In [1]:
# Generate Embeddings from embedding_input

!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.7/345.7 kB 6.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.9 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found exi

In [2]:
# Use the parquet file generated in EDA phase.

import pandas as pd

inScopeMetadata = pd.read_parquet("/kaggle/input/abo-english-metadata-parquet/inScopeMetadata_with_embeddings.parquet")

In [3]:
# check the type before resuming.

type(inScopeMetadata['embedding_vector'].iloc[0])


numpy.ndarray

In [8]:
print(inScopeMetadata.shape)

(122734, 41)


In [9]:
inScopeMetadata.sample(1)

,brand,bullet_point,color,item_id,item_name,item_weight,material,model_name,model_number,product_type,...,item_name_flat,item_weight_flat,material_flat,model_name_flat,model_number_flat,product_type_flat,item_keywords_flat,node_flat,embedding_input,embedding_vector
64145,"[{'language_tag': 'en_IN', 'value': 'Amazon Br...","[{'language_tag': 'en_IN', 'value': '3D Printe...","[{'language_tag': 'en_IN', 'standardized_value...",B07TRX487W,"[{'alternate_representations': None, 'language...","[{'normalized_value': {'unit': 'pounds', 'valu...",None,"[{'language_tag': 'en_IN', 'value': 'Oppo F11 ...",[{'value': 'gz8637-SL40780'}],[{'value': 'CELLULAR_PHONE_CASE'}],...,Amazon Brand - Solimo Designer Multicolor Circ...,50,None,Oppo F11 Pro,gz8637-SL40780,CELLULAR_PHONE_CASE,"mobile cover, back cover, mobile case, phone c...",None,Amazon Brand - Solimo Designer Multicolor Circ...,"[-0.112759896, 0.02776274, 0.032392133, -0.028..."


In [10]:
duplicate_counts = inScopeMetadata['item_id'].value_counts()
duplicate_counts = duplicate_counts[duplicate_counts > 1]
print(duplicate_counts)

item_id
B07WC622LH    28
B0746MMVXW    28
B07797D9MW    27
B073S3R169    26
B01928HSB4    26
              ..
B07GN662BP     7
B07RRWD8QB     7
B07HFTZ8YW     7
B07RR3RFHT     7
B01719EX2S     7
Name: count, Length: 9225, dtype: int64


In [12]:
!pip install faiss-cpu --quiet

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 55.0 MB/s eta 0:00:00:00:0100:01


In [19]:
#Load model and encode

from sentence_transformers import SentenceTransformer

# Load the model (compact + effective)
model = SentenceTransformer('all-MiniLM-L12-v2')

# Generate dense embeddings (512-dimensional vectors)
embedding_list = model.encode(
    inScopeMetadata['embedding_input'].tolist(),
    show_progress_bar=True,
    convert_to_numpy=True
)

Batches:   0%|          | 0/3836 [00:00<?, ?it/s]

In [20]:
# Attach to the dataframe

import numpy as np

# Save as separate column or matrix
inScopeMetadata['embedding_vector'] = list(embedding_list)

In [21]:
import numpy as np

embedding_matrix = np.vstack(embedding_list)

In [22]:
print(embedding_matrix.shape)

(122734, 384)


In [23]:
# Create vector index

import faiss

dim = embedding_matrix.shape[1]
faiss_index = faiss.IndexFlatL2(dim)
faiss_index.add(embedding_matrix)  # 🔥 fast vector search

In [14]:
np.save("embedding_matrix.npy", embedding_matrix)
#embedding_matrix = np.load("embedding_matrix.npy")
faiss.write_index(index, "faiss_index.index")
# index = faiss.read_index("faiss_index.index")

With above copies, we have:

faiss_index.index → FAISS binary index
inScopeMetadata_with_embeddings.parquet → all metadata + embedding inputs
embedding_matrix.npy → optional fallback

In [24]:

# 3. Sample record
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [25]:
import numpy as np
import faiss
import pandas as pd
from collections import OrderedDict

def rerank_faiss_results(query_embedding, faiss_index, metadata_df, top_k=50, final_k=10):
    """
    query_embedding: np.array of shape (384,) from all-MiniLM-L12-v2
    faiss_index: FAISS index object
    metadata_df: DataFrame with a column 'item_id' aligned by FAISS vector index
    top_k: how many closest vectors to fetch initially
    final_k: how many unique item_ids to return
    """

    # Reshape query embedding
    query_embedding = np.array([query_embedding]).astype('float32')

    # Get top_k nearest embeddings
    D, I = faiss_index.search(query_embedding, top_k)
    distances = D[0]
    indices = I[0]

    # Track top unique items
    seen_item_ids = OrderedDict()
    
    for dist, idx in zip(distances, indices):
        item_id = metadata_df.iloc[idx]['item_id']
        if item_id not in seen_item_ids:
            seen_item_ids[item_id] = {
                'item_id': item_id,
                'index': idx,
                'distance': dist,
                'metadata': metadata_df.iloc[idx].to_dict()
            }
        if len(seen_item_ids) >= final_k:
            break

    # Convert to DataFrame or list
    results = list(seen_item_ids.values())
    return results

In [26]:
print(inScopeMetadata.columns.tolist())

['brand', 'bullet_point', 'color', 'item_id', 'item_name', 'item_weight', 'material', 'model_name', 'model_number', 'product_type', 'main_image_id', 'other_image_id', 'item_keywords', 'country', 'marketplace', 'domain_name', 'node', 'style', 'item_dimensions', 'model_year', 'color_code', 'spin_id', '3dmodel_id', 'fabric_type', 'item_shape', 'pattern', 'product_description', 'finish_type', 'brand_flat', 'bullet_point_flat', 'color_flat', 'item_name_flat', 'item_weight_flat', 'material_flat', 'model_name_flat', 'model_number_flat', 'product_type_flat', 'item_keywords_flat', 'node_flat', 'embedding_input', 'embedding_vector']


In [30]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L12-v2')
query = "Shoe"
query_embedding = model.encode(query)

top_results = rerank_faiss_results(query_embedding, faiss_index, inScopeMetadata, top_k=50, final_k=10)

for res in top_results:
    print(res['item_id'], res['distance'], res['metadata']['item_name_flat'])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

B07D451GP9 0.7125936 Amazon Brand - find. Women's Ankle boots
B07WKBW4RR 0.78247225 find. Women's Lace-s-2-46 Open Toe Sandals
B07KMRBT8Q 0.7946671 find. Round Toe Block Heel Leather Court, Women’s Closed-Toe Pumps
B0812BSW74 0.79647666 find. Men's Cupsole Boat Shoe, Blue Navy Suede, women 2
